# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
pd.options.display.mpl_style = 'default';

#set figsize for all figures in notebook
plt.rcParams["figure.figsize"] = [12.0, 9.0];

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 
              'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 
              'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int,
              'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [4]:
sales = pd.read_csv('./data/kc_house_data.csv', dtype=dtype_dict)

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [5]:
train_data = pd.read_csv('./data/kc_house_train_data.csv')
test_data = pd.read_csv('./data/kc_house_test_data.csv')
# train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
from sklearn import linear_model

In [8]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = linear_model.LinearRegression()
example_model.fit(train_data[example_features].values, train_data['price']);
# example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
#                                                   validation_set = None)

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [9]:
print "Example model"
print "Slope: ", example_model.coef_[0]
print "Intercept: ", example_model.intercept_
# example_weight_summary = example_model.get("coefficients")
# print example_weight_summary

Example model
Slope:  315.406690615
Intercept:  87912.865815


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [11]:
example_predictions = example_model.predict(train_data[example_features].values)
print example_predictions[0] # should be 271789.505878

271789.26538


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [38]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    
    # Then compute the residuals/errors
    residuals = outcome - predictions
    
    # Then square and add them up
    RSS = (residuals*residuals).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [40]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features].values, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.73761940583e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [14]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [18]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [17]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data.apply(lambda x: x['bedrooms']*x['bathrooms'], axis = 1)
test_data['bed_bath_rooms'] = test_data.apply(lambda x: x['bedrooms']*x['bathrooms'], axis = 1)
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data.apply(lambda x: x['lat'] + x['long'], axis = 1)
test_data['lat_plus_long'] = test_data.apply(lambda x: x['lat'] + x['long'], axis = 1)

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [19]:
print test_data['bedrooms_squared'].mean()

12.4466777016


In [20]:
print test_data['bed_bath_rooms'].mean()

7.50390163159


In [21]:
print test_data['log_sqft_living'].mean()

7.55027467965


In [22]:
print test_data['lat_plus_long'].mean()

-74.653333554


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [23]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [24]:
# Learn the three models: (don't forget to set validation_set = None)
model1 = linear_model.LinearRegression()
model2 = linear_model.LinearRegression()
model3 = linear_model.LinearRegression()

model1.fit(train_data[model_1_features].values, train_data['price'])
model2.fit(train_data[model_2_features].values, train_data['price'])
model3.fit(train_data[model_3_features].values, train_data['price'])

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [35]:
# Examine/extract each model's coefficients:
print "Model 1"
for pair in (list(zip(model1.coef_, model_1_features))):
    print pair[1], pair[0]
    
print "\nModel 2"
for pair in (list(zip(model2.coef_, model_2_features))):
    print pair[1], pair[0]

Model 1
sqft_living 312.258646273
bedrooms -59586.5331536
bathrooms 15706.7420827
lat 658619.263931
long -309374.351268

Model 2
sqft_living 306.610053459
bedrooms -113446.36807
bathrooms -71461.3082928
lat 654844.629503
long -294298.969138
bed_bath_rooms 25579.6520008


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [42]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model1_train_rss = get_residual_sum_of_squares(model1, train_data[model_1_features].values, train_data['price'])
print "Model 1 RSS: ", model1_train_rss

model2_train_rss = get_residual_sum_of_squares(model2, train_data[model_2_features].values, train_data['price'])
print "\nModel 2 RSS: ", model2_train_rss

model3_train_rss = get_residual_sum_of_squares(model3, train_data[model_3_features].values, train_data['price'])
print "\nModel 3 RSS: ", model3_train_rss

Model 1 RSS:  9.6787996305e+14

Model 2 RSS:  9.58419635074e+14

Model 3 RSS:  9.03436487011e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [45]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model1_test_rss = get_residual_sum_of_squares(model1, test_data[model_1_features].values, test_data['price'])
print "Model 1 RSS: ", model1_test_rss

model2_test_rss = get_residual_sum_of_squares(model2, test_data[model_2_features].values, test_data['price'])
print "\nModel 2 RSS: ", model2_test_rss

model3_test_rss = get_residual_sum_of_squares(model3, test_data[model_3_features].values, test_data['price'])
print "\nModel 3 RSS: ", model3_test_rss

Model 1 RSS:  2.25500469795e+14

Model 2 RSS:  2.23377462976e+14

Model 3 RSS:  2.59236181838e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.